In [47]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
from config import password

In [40]:
disaster_data = "static/data/disasterData.csv"
disaster_data_df = pd.read_csv(disaster_data)
disaster_data_df["incidentType"].value_counts()

Severe Ice Storm    780
Severe Storms       103
Flood                63
Tornado              24
Fire                 15
Snow                  5
Mud/Landslide         4
Other                 1
Name: incidentType, dtype: int64

In [42]:
disaster_data_df_cleaned = disaster_data_df[['designatedArea','state','lat','lon','fyDeclared','declarationDate','incidentType','declarationTitle']].copy()

In [43]:
#create date only from time column
time_split = disaster_data_df_cleaned['declarationDate'].str.split('T',expand=True)

disaster_data_df_cleaned['date'] = time_split[0]
disaster_data_df_cleaned.head()

,designatedArea,state,lat,lon,fyDeclared,declarationDate,incidentType,declarationTitle,date
0,Wayne County,MI,42.268241,-83.284417,2021,2021-07-15T10:41:00.000Z,Severe Storms,"SEVERE STORMS, FLOODING, AND TORNADOES",2021-07-15
1,Washtenaw County,MI,42.258204,-83.844085,2021,2021-07-15T10:41:00.000Z,Severe Storms,"SEVERE STORMS, FLOODING, AND TORNADOES",2021-07-15
2,Statewide,WA,38.894992,-77.036558,2021,2021-07-14T18:36:00.000Z,Fire,RED APPLE FIRE,2021-07-14
3,Klamath County,OR,42.605257,-121.739544,2021,2021-07-10T20:22:00.000Z,Fire,BOOTLEG FIRE,2021-07-10
4,Spokane County,WA,47.535496,-117.407467,2021,2021-07-06T00:05:00.000Z,Fire,ANDRUS FIRE,2021-07-06


In [45]:
disaster_data_df_cleaned = disaster_data_df_cleaned[['designatedArea','state','lat','lon','fyDeclared','date','incidentType','declarationTitle']].copy()

In [46]:
disaster_data_df_cleaned.rename(columns={'designatedArea': 'county', 'stateName': 'state','fyDeclared': 'year'})

,county,state,lat,lon,year,date,incidentType,declarationTitle
0,Wayne County,MI,42.268241,-83.284417,2021,2021-07-15,Severe Storms,"SEVERE STORMS, FLOODING, AND TORNADOES"
1,Washtenaw County,MI,42.258204,-83.844085,2021,2021-07-15,Severe Storms,"SEVERE STORMS, FLOODING, AND TORNADOES"
2,Statewide,WA,38.894992,-77.036558,2021,2021-07-14,Fire,RED APPLE FIRE
3,Klamath County,OR,42.605257,-121.739544,2021,2021-07-10,Fire,BOOTLEG FIRE
4,Spokane County,WA,47.535496,-117.407467,2021,2021-07-06,Fire,ANDRUS FIRE
...,...,...,...,...,...,...,...,...
990,Hidalgo County,TX,26.353026,-98.216445,2021,2021-02-14,Severe Ice Storm,SEVERE WINTER STORM
991,Hill County,TX,31.959509,-97.088165,2021,2021-02-14,Severe Ice Storm,SEVERE WINTER STORM
992,Hood County,TX,32.410756,-97.828282,2021,2021-02-14,Severe Ice Storm,SEVERE WINTER STORM
993,Hopkins County,TX,33.136825,-95.565658,2021,2021-02-14,Severe Ice Storm,SEVERE WINTER STORM


In [ ]:
connection_string = f"postgres:{password}@localhost:5432/disaster_db"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
disaster_data_df_cleaned.to_sql(name='disaster', con=engine, if_exists='append', index=True)